In [174]:
import time
import random

In [74]:
def make_list(file):
    lst=[]
    f = open(file, "r")
    for i in f:
        lst.append([int(x) for x in i.split()])
        
    return lst

def move(board, piece):
    copy_board = board.copy()
    index = board.index(piece)

    # switch 0 piece with piece
    temp = copy_board[index]
    copy_board[copy_board.index(0)] = temp
    copy_board[index] = 0

    return copy_board

def get_move_cost(piece, board):    
    zero_index = board.index(0)
    piece_index = board.index(piece)
    
    #if <4, top row
    #if >=4, top row
    row_length = len(board)/2
    
    # 0 and piece are on the same row
    if (zero_index < row_length and piece_index < row_length) or (zero_index >= row_length and piece_index >= row_length):
        # left / right
        if abs(zero_index - piece_index) == 1:
            return 1
            
        # wrap
        if abs(zero_index - piece_index) == (row_length - 1):
            return 2
    
    # 0 and piece are on different rows
    else:
        # up / down
        if abs(zero_index - piece_index) == row_length:
            return 1
        
        # diagonal
        if abs(zero_index - piece_index) == 1 or abs(zero_index - piece_index) == 3 or abs(zero_index - piece_index) == 5 or abs(zero_index - piece_index) == 7:
            return 3

    # invalid move
    return -1

        
def check_solved(board):
    solutions = [[1,2,3,4,5,6,7,0], [1,3,5,7,2,4,6,0]]
    return board in solutions

    
def possible_moves(board):
    moves = []
    for i in board:
        if get_move_cost(i, board) != -1:
            moves.append(i)
    return moves
    
def heuristics(h_type, board):
    sol = [[1,2,3,4,5,6,7,0], [1,3,5,7,2,4,6,0]]
    
    # naive heuristic
    if h_type == 0:
        if check_solved(board):
            return 0
        else:
            return 1
    
    # manhattan distance heuristic
    if h_type == 1:
        manhattan = [0,0]
        for s in range(0,len(sol)):
            for i in board:
               manhattan[s] += abs(board.index(i) - sol[s].index(i))
        return min(manhattan)
        
    if h_type == 2:
        h = 0
        for s in range(0,len(sol)):
            for i in board:
               manhattan[s] += abs(board.index(i) - sol[s].index(i))
        return h
    

In [183]:
def gbfs(heuristic_type, puzzles):
    iteration = -1
    for board in puzzles:
        
        iteration += 1
        
        open_lst = []
        closed_lst = []
        current = board
        sol_output = '0 0 {}\n'.format(str(current).strip('[]').replace(',', ''))
        search_output = 'f(n) g(n) h(n) path\n'
        sum_cost = 0
        
        
        # time limit in case solution not found
        time_limit = 1.0
        start_time, end_time = time.time(), time.time()

        while not check_solved(current) and (end_time - start_time < time_limit):

            # add node to closed list
            closed_lst.append(current)

            # find all possible moves and add to open list
            open_lst = possible_moves(board)

            # min heuristic and piece to move for best node
            min_heuristic = 1000
            best_piece = open_lst[0]
            for i in open_lst:
                next_board = move(current, i)

                # if board not in closed list and has better heuristic, make it next node
                if next_board not in closed_lst and heuristics(heuristic_type, next_board) < min_heuristic:
                    best_piece = i
                    min_heuristic = heuristics(heuristic_type, next_board)
            
            #data for search/visited nodes as fn gn hn path
            search_line = '{} 0 {} {}\n'.format(heuristics(heuristic_type, next_board),
                                                heuristics(heuristic_type, next_board),
                                               str(next_board).strip('[]').replace(',', ''))
            search_output += search_line
            # move to node with lowest heuristic
            current = move(current, best_piece)
            
            #data to write to file if solved
            cost = get_move_cost(best_piece, current)
            sum_cost += cost
            line = '{} {} {}\n'.format(best_piece, cost, str(current).strip('[]').replace(',', ''))
            sol_output += line

            # update timer
            end_time = time.time()
        #create file for solution output
        solution_filename = '{}{}{}{}'.format(iteration, '_gbfs-h', heuristic_type,'_solution.txt')
        search_filename = '{}{}{}{}'.format(iteration, '_gbfs-h', heuristic_type,'_search.txt')
        
        with open(search_filename, 'w') as f:
                f.write(search_output)

        if check_solved(current):
            print("Solved {}".format(current))
            #write to file
            cost_time_line = '{} {}'.format(sum_cost, round((end_time - start_time), 2))
            sol_output += cost_time_line
            with open(solution_filename, 'w') as f:
                f.write(sol_output)
            print(search_output)
        else:
            with open(solution_filename, 'w') as f:
                f.write("No Solution")




In [172]:
puzzles = make_list('samplePuzzles.txt')

gbfs(1,puzzles)

In [211]:
#generating 50 random puzzles
rand_puzzles = []
for i in range(0, 50):
    sol = [1,2,3,4,5,6,7,0]
    random.shuffle(sol)
    rand_puzzles.append(sol)

50


In [210]:
gbfs(1,rand_puzzles)

Solved [1, 3, 5, 7, 2, 4, 6, 0]
f(n) g(n) h(n) path
10 0 10 1 5 7 4 2 6 3 0
6 0 6 1 5 3 4 2 6 7 0
8 0 8 1 5 3 4 2 6 0 7
8 0 8 1 5 3 4 2 7 0 6
10 0 10 1 5 3 4 2 0 7 6
10 0 10 1 5 3 4 2 0 6 7
6 0 6 1 5 3 7 2 0 6 4
10 0 10 1 5 3 0 2 7 6 4
10 0 10 1 5 3 0 2 4 6 7
10 0 10 1 5 3 0 2 4 7 6
10 0 10 1 5 3 0 2 7 4 6
12 0 12 1 5 3 0 2 6 4 7
12 0 12 1 5 3 0 2 6 7 4
6 0 6 1 5 3 7 2 6 0 4
10 0 10 1 5 3 6 2 7 0 4
10 0 10 1 5 3 6 2 4 0 7
4 0 4 1 5 3 7 2 4 0 6
6 0 6 1 7 3 5 2 4 0 6
12 0 12 1 0 3 5 2 4 7 6
12 0 12 1 0 3 5 2 4 6 7
12 0 12 1 0 3 5 2 7 6 4
14 0 14 1 0 3 5 2 6 7 4
14 0 14 1 0 3 5 2 6 4 7
12 0 12 1 0 3 5 2 7 4 6
12 0 12 1 0 3 7 2 5 4 6
12 0 12 1 0 3 4 2 5 7 6
12 0 12 1 0 3 4 2 5 6 7
14 0 14 1 0 3 4 2 7 6 5
12 0 12 1 0 3 7 2 4 6 5
16 0 16 1 0 3 6 2 4 7 5
16 0 16 1 0 3 6 2 4 5 7
12 0 12 1 0 3 7 2 4 5 6
14 0 14 1 0 3 4 2 7 5 6
12 0 12 1 0 3 4 2 6 5 7
12 0 12 1 0 3 4 2 6 7 5
12 0 12 1 7 3 4 2 6 0 5
12 0 12 1 4 3 7 2 6 0 5
8 0 8 1 4 3 5 2 6 0 7
6 0 6 1 4 3 5 2 6 7 0
8 0 8 1 4 3 5 2 7 6 0
10 0 10 